# <center>Eye Tracking Project</center>

## Table of Contents

1. [Import Requirements](#Import-Requirements)
2. [Context Information](#Context-information)
3. [Paper 1: IPA](#The-Index-of-Pupillary-Activity)
    1. [Version 1](#Version-1) Hardcoded Version 

## Import Requirements
These are the required packages to make the program work, I tried to limit it to only what was found within the original pseudocode with only a few additional ones where I thought they would be best used.

In [66]:
import math
import numpy as np
import pandas as pd
import pywt

from matplotlib import pyplot as plt
from scipy.integrate import quad

## Context information
I will be trying to implement three pupillometric measures from Duchowski's papers regarding: 
- 1) Index of Pupillary Activity (IPA), 
- 2) Low/High Index of Pupillary Activity (LHIPA), 
- 3) Reat-Time-Index of Pupillary Activity (RIPA) 

Which are all loosely based on Marshall's Index of Cognitive Activity (ICA) from 2002.

## The Index of Pupillary Activity
The paper Duchowski et al. (2018) is the basis for this and it is the first iteration of their eye-tracked measure of the requency of pupil diameter oscillation. What this paper achieves is creating/replicating the Index of Cognitive Activity (ICA) and in general improving upon it without necessarily being copyrighted. 


## Version 1
This version was created before I was aware that PyWavelet could do 90% of what I needed for the code. I kept this part here as part of the coding process but it is ultimately left out of the final version and is only found in the PDF version of the code.

### IPA Implementation
There are roughly six formulas that are used within the paper to calculate IPA. Below we will go through each one and how they are implemented, what their variables are, and how they connect to one another. 


#### Formula 1: Dyadic Wavelet Function
We will do this with the following variables:
- $x(t)$: pupil diameter signal over time
- $\psi(t)$: the mother wavelet function 
    - (Daubechies-4 if 60Hz or Daubechies-16 if 250Hz)
- $j$: dilation parameter, integers from a set that can represent any number
- $k$: translation parameter, integers from a set that can represent any number
- $2^{j/2}$: Scaling factor w.r.t. time domain
- $\psi(2^j t - k)$: Shifted and scaled wavelet function

$$
\psi_{j,k}(t) = 2^{j/2} \psi(2^j t - k), \quad \text{where } j, k \in \mathbb{Z} \qquad \text{(0)}
$$

In [61]:
# Notable concerns:
# - None

def check_for_integers(list_given):
    """
    Function Description: This will check if the list provided is made only of integers
    
    Parameters:
        list_given = a list
        
    Returns:
        Verification that the list is only Integers
    """
    try:
        import numpy as np
    except ImportError:
        raise ImportError("Numpy is not imported.")
    return np.all(np.array(list_given, dtype=np.int64)) | np.all(np.array(list_given, dtype=np.int32))


# Notable concerns:
# - We will need more than one input feature as we don't have j or k
# - We are unsure if we're using 60Hz or 250Hz so we'll assume 60Hz

def compute_dyadic_wavelet(t, j, k, w):
    """
    Function Description: This will calculate the dyadic wavelet transformation.
    
    Parameters:
        t = pupil diameter (float)
        j = dilation parameter (integer)
        k = translation parameter (integer)
        w = wavelet (string)
    
    Returns: 
        This returns the calculation of the dyadic wavelet transformation.
    
    """
    try:
        import pywt
    except ImportError:
        raise ImportError("Py Wavelet is not imported.")
        
    try:
        # This will make sure that 𝑗,𝑘∈ℤ
        if not (check_for_integers(j) and check_for_integers(k)):
            raise ValueError("j and k must be lists of integers.")
        
        # This will make sure that we're only inputting the correct db
        if not (isinstance(w, str) and w in ["db4", "db16"]):
            raise ValueError("w must be either Daubechies-4 or Daubechies-16")
        
    
        # Determine what wavelet we are using
        wf = pywt.Wavelet(w)

        # Calculation of the Scaling Factor and Translation Factor
        psi_t = 2**(j/2) * wf.wavefun(level=2)[0]  
        interior_calc = psi_t(2**j * t - k) 

        return interior_calc
    
    except ValueError as ve:
        print("ValueError:", ve)
        return None

#### Formula 2: Integral Transformation of Wavelet Coefficients
This step is far more complicated as it goes through several iterations of it within the Duchowski et al. (2018) paper, but we'll mainly focus on the steps taht we can complete here.

(For the sake of space, repeated variables will not be given descriptions)

##### Decomposition of the Wavelet Analysis w.r.t. coefficients
This formula gives us the wavelet function as a linear combination that allows us to do wavelet decomposition. 

Variable representation:
- $L^2(\mathbb{R})$: Noting that $x(t)$ is a square-integrable function
    - (meaning it won't become infinitely large when squared or integrated)
- $\sum_{j,k = -\infty}^{\infty}$: represents the sum over all possible combinations of j and k that are integers.
- $c_{j,k}$: wavelet coefficients of $\psi_{j,k}(t)$
- $x(t)$: pupil diameter signal
- $\psi_{j,k}(t)$: the mother wavelet function 
    - (Daubechies-4 if 60Hz or Daubechies-16 if 250Hz)


$$
x \in L^2(\mathbb{R}): x(t) = \sum_{j,k = -\infty}^{\infty} c_{j,k} \psi_{j,k}(t), \quad j,k \in \mathbb{Z} \qquad \text{(1)}
$$

##### Inner Product Calculation of the Wavelet Coefficients
This formula computes the wavelet coefficients by taking the inner product of the input $x(t)$ and tells us how well the function performs at different scales and translations.

Variable representation:
- $c_{j,k}$: already noted
- $\int_{-\infty}^{\infty}$: Integral of the formula, represents area under the curve
- $x(t)$: already noted
- $\overline{\psi_{j,k}(t)}$: We are taking the complex conjugate of our previous wavelet function. 
    - (this ensures that the result is a real value)
- $dt$: The integration of the variable t

$$
c_{j,k} = \int_{-\infty}^{\infty} x(t) \overline{\psi_{j,k}(t)} dt, \quad x \in L^2(\mathbb{R}), \quad j,k \in \mathbb{Z} \qquad \text{(2)}
$$

##### Substitution Property for Formula 1 and Formula 2
This formula incorporates Formula 1 into Formula 2 where the $\psi$ was located.

Variables inlcuded from Formula 1:
- $j$: dilation parameter, integers from a set that can represent any number
- $k$: translation parameter, integers from a set that can represent any number
- $2^{j/2}$: Scaling factor w.r.t. time domain
- $\psi(2^j t - k)$: Shifted and scaled wavelet function

$$
c_{j,k} = 2^{j/2} \int_{-\infty}^{\infty} x(t) \overline{\psi(2^j t - k)} \, dt, \quad x \in L^2(\mathbb{R}), \quad j,k \in \mathbb{Z}
\qquad \text{(3)}
$$

In [60]:
# Notable concerns:
# - We will need more than one input feature as we don't have j or k
# - We are unsure if we're using 60Hz or 250Hz so we'll assume 250Hz



def compute_wavelet_coefficients(t, j2, k2, w):
    """
    Function Description: Compute the wavelet coefficients c_{j,k} for the given signal x(t).
    
    Parameters:
        t = pupil diameter (float)
        j2 = dilation parameter (integer), name is j2 to differentiate from j further within
        k2 = translation parameter (integer), name is k2 to differentiate from k further within
        w = wavelet (string)
        
    Returns:
        c_jk: A dictionary containing the computed wavelet coefficients c_{j,k}
    """
    
    try:
        from scipy.integrate import quad
    except ImportError:
        raise ImportError("Scipy.integrate is not imported.")
        
    try:
        # This makes sure that j2 and k2 are lists of integers
        if not (check_for_integers(j_values) and check_for_integers(k_values)):
            raise ValueError("j and k must be lists of integers.")

        # This will make sure that we're only inputting the correct db
        if not (isinstance(w, str) and w in ["db4", "db16"]):
            raise ValueError("w must be either Daubechies-4 or Daubechies-16")
       
        # Initialize an empty dictionary for the wavelet coefficients
        c_jk = {}

        # Computation of the wavelet coefficient for both j and k
        for j in j2:
            for k in k2:
                # Compute the integrand for the inner product <x(t), ψ(j, k, t)>
                compute_integrand = lambda t: x(t) * np.conj(dyadic_wavelet(t, j, k, w))
                
                # Computation of the Inner Product, needs Scipy
                inner_product, _ = quad(compute_integrand, -np.inf, np.inf)

                # Creates it into a wavelet coefficient
                c_jk[(j, k)] = 2**(j/2) * inner_product

        return c_jk
    
    except ValueError as ve:
        print("ValueError:", ve)
        return None

##### The Wavelet Coefficients in their final form
This formula gives us the similarity between the signal and the wavelet function at each scale and position.

Variable representation:
- $\{W_\psi x(t)\}(j, k)$: The wavelet transformation of $x(t)$ using $\psi$
- $\langle x(t), \psi_{j,k}(t) \rangle$: The inner product reprsented in another way from formula (3)

$$
= \{W_\psi x(t)\}(j, k) = \langle x(t), \psi_{j,k}(t) \rangle
\qquad \text{(4)}
$$

# Version 2
At this point, I had finally read through the majority of PyWavelet and realized that I spent probably 20+ hours trying to understand and to hard code items that were already calculated within the library itself because I didn't fully understand it.



In [ ]:
class IPA:
    def __init__(self, pupil_diam_data, wavelet):
        self.pupil_diam_data = pupil
        self.wavelet = wavelet
        self.scaling = wavelet
        
    def wavelet_decomposition(self):
        

In [62]:
import numpy as np
import pywt

class PupilAnalyzer:
    def __init__(self, pupil_data):
        self.pupil_data = pupil_data
        self.wavelet = 'db4'  # Default wavelet
        self.scaling_function = 'db4'  # Default scaling function
        self.sampling_rate = 1  # Placeholder value, actual sampling rate should be known
    
    def preprocess_data(self):
        # Preprocessing step to remove data around blinks
        # Placeholder implementation, actual blink detection not included
        pass  # You can implement actual preprocessing steps here
    
    def wavelet_decomposition(self):
        # Wavelet decomposition
        self.wavelet_coeffs = pywt.wavedec(self.pupil_data, self.wavelet)
    
    def scaling_function_analysis(self):
        # Scaling function analysis using Stationary Wavelet Transform (SWT)
        self.scaling_coeffs = pywt.swt(self.pupil_data, self.scaling_function)
    
    def compute_ipa_from_coeffs(self, coeffs):
        # Counting abrupt discontinuities in coefficients
        abrupt_changes = 0
        for c in coeffs:
            diffs = np.diff(c)
            abrupt_changes += np.count_nonzero(diffs)
        
        # Computing IPA as frequency of abrupt discontinuities
        time_duration = len(coeffs[-1]) / self.sampling_rate
        ipa = abrupt_changes / time_duration
        
        return ipa
    
    def analyze(self):
        self.preprocess_data()
        self.wavelet_decomposition()
        self.scaling_function_analysis()
        
        ipa_from_wavelet = self.compute_ipa_from_coeffs(self.wavelet_coeffs)
        ipa_from_scaling = self.compute_ipa_from_coeffs(self.scaling_coeffs)
        
        return ipa_from_wavelet, ipa_from_scaling

# Example usage
pupil_data = np.random.rand(1000)  # Example pupil diameter data
analyzer = PupilAnalyzer(pupil_data)
ipa_from_wavelet, ipa_from_scaling = analyzer.analyze()
print("Index of Pupillary Activity (IPA) from wavelet analysis:", ipa_from_wavelet)
print("Index of Pupillary Activity (IPA) from scaling function analysis:", ipa_from_scaling)


Index of Pupillary Activity (IPA) from wavelet analysis: 2.0636182902584492
Index of Pupillary Activity (IPA) from scaling function analysis: 2997.0


# Tester

In [67]:
df1 = pd.read_csv('left_eye_pupil_measures.csv')
df1.head()

,world_timestamp,world_index,eye_id,diameter
0,260229.466438,2,0,0.0
1,260229.474838,2,0,0.0
2,260229.483238,2,0,0.0
3,260229.491638,3,0,0.0
4,260229.500038,3,0,0.0


In [64]:
import math
import numpy as np
import pywt

class PupilAnalyzer:
    def __init__(self, pupil_data):
        self.pupil_data = pupil_data
        self.sampling_rate = 1  # Placeholder value, actual sampling rate should be known
        self.wavelet = 'sym16'  # Wavelet for DWT
        self.level = 2  # Number of levels for DWT

    def wavelet_decomposition(self):
        # Perform 2-level DWT of pupil diameter signal
        try:
            coeffs = pywt.wavedec(self.pupil_data, self.wavelet, mode='per', level=self.level)
            return coeffs
        except ValueError:
            return None

    def normalize_coefficients(self, coeffs):
        # Normalize DWT coefficients
        for i in range(1, self.level + 1):
            coeffs[i] = [x / math.sqrt(2 ** i) for x in coeffs[i]]
        return coeffs

    def modmax(self, d):
        # Compute signal modulus
        m = [math.fabs(x) for x in d]
        # Compute local maxima
        t = [0.0] * len(d)
        for i in range(len(d)):
            ll = m[i - 1] if i >= 1 else m[i]
            oo = m[i]
            rr = m[i + 1] if i < len(d) - 1 else m[i]
            if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
                # Compute magnitude
                t[i] = math.sqrt(d[i] ** 2)
        return t

    def thresholding(self, coeffs):
        # Thresholding using universal threshold
        cD2m = self.modmax(coeffs[-1])  # Assuming the last level contains detail coefficients
        λuniv = np.std(cD2m) * math.sqrt(2.0 * np.log2(len(cD2m)))
        cD2t = pywt.threshold(coeffs[-1], λuniv, mode="hard")
        coeffs[-1] = cD2t
        return coeffs

    def compute_ipa(self, coeffs):
        # Compute IPA
        ctr = sum(math.fabs(x) > 0 for x in coeffs[-1])
        tt = len(self.pupil_data) / self.sampling_rate
        ipa = float(ctr) / tt
        return ipa

    def analyze(self):
        # Perform analysis
        coeffs = self.wavelet_decomposition()
        if coeffs is None:
            return None
        coeffs = self.normalize_coefficients(coeffs)
        coeffs = self.thresholding(coeffs)
        ipa = self.compute_ipa(coeffs)
        return ipa

# Example usage
pupil_data = np.random.rand(1000)  # Example pupil diameter data
analyzer = PupilAnalyzer(pupil_data)
ipa = analyzer.analyze()
print("Index of Pupillary Activity (IPA):", ipa)


Index of Pupillary Activity (IPA): 0.0


In [70]:
analyzer = PupilAnalyzer(df1.diameter)
ipa = analyzer.analyze()
print("Index of Pupillary Activity (IPA):", ipa)

Index of Pupillary Activity (IPA): 0.0030872859949480774


In [ ]:
class PyPil:
    def __init__(self, pupil_data, wavelet="sym16", level = "2"):
        self.pupil_data = pupil_data
        self.wavelet = wavelet
        self.level = level
    